Premier exemple de classification binaire: prédire la détention du baccalauréat dans le recensement 

# Présentation
Dans ce tutoriel, nous allons voir comment utiliser une forêt aléatoire pour prédire si un individu a obtenu le baccalauréat à partir de données du recensement de la population (Insee). Nous présentons les étapes de préparation des données, d'entraînement du modèle, d'optimisation des hyperparamètres et d'interprétation des résultats.

Le jeu de données contient des informations individuelles issues du recensement, telles que l'âge, le niveau d'éducation, la situation professionnelle, etc.

L'objectif ici est de prédire si un individu a obtenu le baccalauréat en fonction des autres caractéristiques observées.


# Préparation de l'environnement

## Importation des bibliothèques nécessaires

In [ ]:
# Importation des bibliothèques pour la manipulation des données
import os
import pandas as pd
import numpy as np

# Bibliothèques pour la visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Bibliothèques pour le traitement des données
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split

# Bibliothèques pour le modèle et l'évaluation
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    ConfusionMatrixDisplay
)

# Pour mesurer le temps d'exécution
import time

## Configuration du répertoire de travail


In [ ]:
# Définir le répertoire du projet 
os.chdir('/home/onyxia/work/methodes_ensemblistes_notebooks')

# Vérification
current_dir = os.getcwd()
print("Répertoire de travail :", current_dir)


# Chargement et aperçu des données

In [ ]:
# Fonction pour lire les fichiers Parquet
def read_parquet_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pq.read_table(local_path).to_pandas()
            print(f"Données Parquet chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier Parquet : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

# Fonction pour lire les fichiers CSV
def read_csv_data(local_path):
    if os.path.exists(local_path):
        try:
            data = pd.read_csv(local_path, sep=';')
            print(f"Données CSV chargées avec succès depuis : {local_path}")
            return data
        except Exception as e:
            print(f"Erreur lors de la lecture du fichier CSV : {e}")
    else:
        print(f"Le fichier n'existe pas : {local_path}")
    return None

In [ ]:
# Charger les données si elles ne sont pas déjà en mémoire
data_census_individuals = read_parquet_data("data/data_census_individuals.parquet", "data_census_individuals", force_reload=True)

# Charger la documentation
doc_census_individuals = read_csv_data("documentation/doc_census_individuals.csv", "doc_census_individuals", force_reload=True)

In [ ]:
# Afficher les premières lignes du jeu de données
data_census_individuals.head()

# Préparation des données

## Echantillonnage des données
Compte tenu de l'objectif purement pédagogique de ce tutoriel, nous tirons un échantillon aléatoire (0,5 %), représentatif de l'ensemble des données initiales, afin d'accélérer les calculs dans les sections suivantes.

In [ ]:
# Échantillonner les données (1/200)
data_sample = data_census_individuals.sample(frac=1/200, random_state=123)

### Suppression de variables redondantes ou contenant trop de modalités

In [ ]:
# Suppression des variables liées à l'âge (autres que AGED) et des variables non pertinentes ou avec trop de modalités (pour commencer)
columns_to_drop = [
    'AGER20', 'AGEREV', 'AGEREVQ', 'ANAI', 'TRIRIS', 'IRIS', 'DNAI',
    'DEPT', 'ARM', 'CANTVILLE', 'NUMMI', 'IPONDI'
]
data_clean = data_sample.drop(columns=columns_to_drop)

In [ ]:
# Voir la répartition des classes de diplôme
print(data_clean['DIPL'].isna().sum())  # Vérification des valeurs manquantes
print(data_clean['DIPL'].value_counts())  # Distribution des classes

## Création de la variable cible : une indicatrice qui vaut 1 pour les détenteurs du baccalauréat

In [ ]:
# Préciser l'odre des catégories de DIPL (ZZ suivi des valeurs numériques croissantes)
categories = ['ZZ'] + [f"{i:02}" for i in range(1, 20)]
data_clean['DIPL'] = pd.Categorical(data_clean['DIPL'], categories=categories, ordered=True)

# Créer la variable binaire 'bac'
data_clean['bac'] = (data_clean['DIPL'] > '13').astype(int)

# Afficher le résultat
print(data_clean[['DIPL', 'bac']].head(20))
print(data_clean['bac'].value_counts())  # Distribution des classes

## Préparation des variables explicatives (features)

In [ ]:
# Supprimer la colonne 'DIPL' du DataFrame (prédicteur parfait du baccalauréat)
data_clean = data_clean.drop(columns=['DIPL'])

In [ ]:
# Séparer les données en features (X) et la variable cible (y : bac)
X = data_clean.drop(columns=['bac'])
y = data_clean['bac']

## Identification des variables catégorielles et numériques

In [ ]:
# Identifier les variables catégorielles
categorical_cols = X.select_dtypes(include=['category']).columns

# Identifier les colonnes numériques
numeric_cols = X.select_dtypes(exclude=['category'])


## Encodage des variables catégorielles
Les variables catégorielles sont transformées en variables numériques grâce au One-Hot Encoding.

In [ ]:
# Encodage "one-hot" des variables catégorielles
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)

# Appliquer le One-Hot Encoding sur les variables catégorielles
X_encoded = encoder.fit_transform(X[categorical_cols])

# Convertir les colonnes encodées en DataFrame avec les noms des catégories
X_encoded = pd.DataFrame(
    X_encoded, 
    index=X.index, 
    columns=encoder.get_feature_names_out(categorical_cols)
)

## Rassembler les variables numériques et catégorielles encodées dans une même table
Nous réunissons toutes les variables explicatives (catégorielles encodées et numériques) dans X.

In [ ]:
# Concaténer les colonnes numériques et encodées (uniquement si les colonnes numériques ne sont pas vides)
if not numeric_cols.empty:
    X = pd.concat([numeric_cols, X_encoded], axis=1)
else:
    X = X_encoded

In [ ]:
# Vérifier les dimensions finales
print(f"Dimensions de X : {X.shape}")
print(f"Nombre de variables numériques : {len(numeric_cols)}")
print(f"Nombre de variables catégorielles encodées : {X_encoded.shape[1]}")

# Dimensions avant et après l'encodage
print(f"Dimensions avant encodage : {X.shape[0]} lignes, {categorical_cols.shape[0]} variables catégorielles")

## Division des données en ensemble d'entraînement et de test
La stratification assure que la proportion des classes est la même dans les ensembles d'entraînement et de test.

In [ ]:
# Diviser les données en ensembles d'entraînement (80%) et de test (20%) avec stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=123, stratify=y
)

# Une première random forest avec des hyperparamètres "standards" (non optimisés)
- n_estimators : Nombre d'arbres dans la forêt.
- max_features : Nombre de variables à considérer pour déterminer le meilleur split.
- min_samples_leaf : Nombre minimum d'échantillons dans une feuille terminale.

In [ ]:
# Un modèle Random Forest pour une classification avec des hyperparamètres de base
rf_model = RandomForestClassifier(
    n_estimators=300,
    max_features='sqrt',
    min_samples_leaf=100,
    random_state=123
)

## Entraînement du modèle

In [ ]:
# Entraînement du modèle
start_time = time.time()
rf_model.fit(X_train, y_train)
elapsed_time = time.time() - start_time
print(f"Temps d'exécution du modèle Random Forest : {elapsed_time:.2f} secondes")

## Evaluation sur le jeu de test

La matrice de confusion permet de visualiser les erreurs de classification.
Le rapport de classification donne des métriques importantes comme la précision, le rappel et le F1-score.

In [ ]:
# Prédictions sur les données de test
y_pred = rf_model.predict(X_test)

In [ ]:
# Évaluer la performance avec une matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred, labels=rf_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=rf_model.classes_)
disp.plot(cmap="Blues")
plt.title("Matrice de confusion pour la prédiction du niveau de diplôme")
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_test, y_pred))

# Optimisation des hyperparamètres

## Influence du nombre d'arbres (n_estimators)

L'augmentation du nombre d'arbres améliore la stabilité et la précision des estimations. Toutefois, un nombre d'arbre élevé peut considérablement allongés le temps de calcul. 

L'objectif est de trouver un compromis entre stabilité et temps de calcul.

Méthode: On observe comment l'accuracy évolue avec le nombre d'arbres.
Cela permet de déterminer si l'augmentation du nombre d'arbres améliore significativement la performance.

In [ ]:
n_estimators_range = [10, 50, 100, 200, 500, 1000]
scores = []

for n in n_estimators_range:
    
    print(f"Entraînement en cours avec n_estimators = {n}")  # Affichage de la valeur actuelle

    rf_model = RandomForestClassifier(
        n_estimators=n, 
        max_features='sqrt', 
        min_samples_leaf=100, 
        random_state=123,
        n_jobs=-1  # Parallélise la construction des arbres
    )
    score = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy').mean()
    scores.append(score)

plt.plot(n_estimators_range, scores, marker='o')
plt.xlabel('Nombre d\'arbres (n_estimators)')
plt.ylabel('Accuracy')
plt.title('Influence de n_estimators')
plt.grid()
plt.show()


## Influence du nombre de caractéristiques (max_features)

In [ ]:
max_features_options = ['sqrt', 0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.8, None]
scores = []
d = X_train.shape[1]  # Nombre total de caractéristiques dans le jeu de données

# Stocker les résultats pour le tri lors de la visualisation
results = []

for mf in max_features_options:

    print(f"Entraînement en cours avec max_features = {mf} ({num_features} features)")  # Affichage de la valeur actuelle

    # Calculer le nombre de features sélectionnés pour chaque option
    if isinstance(mf, float):  # Si mf est une fraction
        num_features = int(mf * d)
    elif mf == 'sqrt':  # Si mf est 'sqrt'
        num_features = int(np.sqrt(d))
    elif mf == 'log2':  # Si mf est 'log2'
        num_features = int(np.log2(d))
    elif mf is None:  # Si mf est None
        num_features = d
    else:
        num_features = None

    # Entraîner et évaluer le modèle
    rf = RandomForestClassifier(
        n_estimators=100,
        max_features=mf,
        min_samples_leaf=100,
        random_state=123,
        n_jobs=-1  # Parallélise la construction des arbres
    )
    score = cross_val_score(rf, X_train, y_train, cv=5, scoring='accuracy').mean()

    # Ajouter les résultats pour tri ultérieur
    results.append((mf, num_features, score))

# Trier les résultats par ordre croissant du nombre de features
results.sort(key=lambda x: x[1])

# Extraire les labels et scores triés
labels = [f"{mf} ({num_features})" for mf, num_features, _ in results]
scores = [score for _, _, score in results]

# Visualisation
plt.plot(labels, scores, marker='o')
plt.xlabel('max_features (nombre de features)')
plt.ylabel('Accuracy moyenne (validation croisée)')
plt.title('Influence de max_features')
plt.grid(True)
plt.xticks(rotation=45)  # Incliner les labels pour une meilleure lisibilité
plt.show()


## Influence de la taille minimale des feuilles (min_samples_leaf)

Pour de faibles valeurs du nombre minimum d'observations par feuille (noeud terminal), le modèle risque de sur-ajuster les données d'entraînement; pour des valeurs élevées, le modèle devient plus simple, ce qui peut entraîner un sous-ajustement.

L'objectif est donc de trouver un compromis optimal entre ces deux situations.


In [ ]:
# Différentes valeurs de min_samples_leaf à tester
min_samples_leaf_range = [5, 10, 50, 100, 500]
scores = []

for min_samples_leaf in min_samples_leaf_range:
        
    print(f"Entraînement en cours avec min_samples_leaf = {min_samples_leaf}")  # Affichage de la valeur actuelle

    rf_model = RandomForestClassifier(
        n_estimators=300,  # Fixé à 500 arbres
        max_features='sqrt',
        min_samples_leaf=min_samples_leaf,
        random_state=123
    )
    score = cross_val_score(rf_model, X, y, cv=5, scoring='accuracy').mean()
    scores.append(score)

# Tracer les résultats
plt.plot(min_samples_leaf_range, scores, marker='o')
plt.xlabel('Valeurs de min_samples_leaf')
plt.ylabel('Accuracy')
plt.title('Influence de min_samples_leaf')
plt.grid()
plt.show()

## Recherche de la meilleure combinaison d'hyperparamètres: Grid Search pour optimiser plusieurs hyperparamètres à la fois

GridSearchCV de scikit-learn permet d'optimiser de manière simultanée plusieurs hyperparamètres.


### Définition de la grille de paramètres

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 200, 500],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_leaf': [1, 10, 50, 100]
}

### Mise en place du Grid Search

In [ ]:
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=123),
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    verbose=1,
    n_jobs=-1
)

# Exécution du Grid Search
grid_search.fit(X_train, y_train)

### Résultats du Grid Search

In [ ]:
print("Meilleurs paramètres :", grid_search.best_params_)
print("Meilleure précision :", grid_search.best_score_)

# Evaluer la performance du modèle optimisé par Grid Search

## Prédictions et évaluation

In [ ]:
best_model = grid_search.best_estimator_
test_score = best_model.score(X_test, y_test)
print("Précision sur le jeu de test :", test_score)

In [ ]:
# Prédictions sur les données de test
y_pred = best_model.predict(X_test)

In [ ]:
# Évaluer la performance avec une matrice de confusion
conf_matrix = confusion_matrix(y_test, y_pred, labels=best_model.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=best_model.classes_)
disp.plot(cmap="Blues")
plt.title("Matrice de confusion pour la prédiction du niveau de diplôme")
plt.show()

In [ ]:
# Rapport de classification
print(classification_report(y_test, y_pred))

## Comparaison avec le modèle initial

In [ ]:
# Calcul de l'accuracy sur le jeu de test pour les deux modèles
initial_accuracy = rf_model.score(X_test, y_test)
optimized_accuracy = best_model.score(X_test, y_test)

print(f"Accuracy du modèle initial : {initial_accuracy:.4f}")
print(f"Accuracy du modèle optimisé : {optimized_accuracy:.4f}")

# Interprétation du modèle : Importance des variables

## Extraction des importances

In [ ]:
# Importance des variables (Mean Decrease in Impurity)
importance_df = pd.DataFrame({
    'Variable': X.columns,
    'Importance': best_model.feature_importances_
}).sort_values(by='Importance', ascending=False)

## Agrégation des importances pour les variables catégorielles

In [ ]:
# Rassembler l'importance des modalités d'une même variable
def calculate_aggregated_importance(X, feature_importances):
    """
    Agrège l'importance des variables catégorielles encodées en colonnes multiples.

    Args:
        X (pd.DataFrame): Les données d'entraînement (features).
        feature_importances (array): Les importances des variables calculées par le modèle.

    Returns:
        pd.DataFrame: Importance des variables agrégées par nom de variable.
    """
    # Récupérer les noms des colonnes
    feature_names = X.columns
    
    # Initialiser un dictionnaire pour stocker les importances agrégées
    aggregated_importance = {}

    for col in feature_names:
        # Identifier les variables spécifiques pour regrouper correctement
        if col.startswith('STAT_CONJ_'):
            variable = 'STAT_CONJ'
        elif col.startswith('STATR_'):
            variable = 'STATR'
        else:
            # Utiliser le préfixe par défaut basé sur le premier segment avant "_"
            variable = col.split('_')[0]
        
        # Additionner les importances pour chaque variable "parent"
        if variable in aggregated_importance:
            aggregated_importance[variable] += feature_importances[feature_names.get_loc(col)]
        else:
            aggregated_importance[variable] = feature_importances[feature_names.get_loc(col)]

    # Convertir en DataFrame trié par importance
    aggregated_df = pd.DataFrame({
        'Variable': aggregated_importance.keys(),
        'Importance': aggregated_importance.values()
    }).sort_values(by='Importance', ascending=False)

    return aggregated_df

In [ ]:
# Appeler la fonction pour regrouper les importances
aggregated_importance_df = calculate_aggregated_importance(X, best_model.feature_importances_)

# Afficher les résultats
print(aggregated_importance_df)

In [ ]:
# Charger les libellés des variables depuis le dictionnaire
doc_census_individuals_noms_variables = doc_census_individuals[['COD_VAR', 'LIB_VAR']].drop_duplicates()

In [ ]:
# Associer les libellés (descriptions) aux variables d'importance
importance_df_with_labels = aggregated_importance_df.merge(
    doc_census_individuals_noms_variables,
    left_on='Variable',
    right_on='COD_VAR',
    how='left'
)

## Visualisation des variables les plus importantes
La visualisation aide à comprendre quelles variables contribuent le plus à la prédiction.

In [ ]:
# Fonction pour visualiser des n variables les plus importantes
def plot_top_n_variables(importance_df, variable_col='Variable', importance_col='Importance', top_n=10):
    """
    Visualise les n variables les plus importantes à partir d'un DataFrame d'importance.

    Parameters:
    - importance_df (pd.DataFrame): Un DataFrame contenant les colonnes spécifiées.
    - variable_col (str): Le nom de la colonne contenant les noms des variables.
    - importance_col (str): Le nom de la colonne contenant les valeurs d'importance.
    - top_n (int): Le nombre de variables les plus importantes à afficher.
    """
    # Vérifier si les colonnes existent
    if variable_col not in importance_df.columns or importance_col not in importance_df.columns:
        raise ValueError(f"Les colonnes '{variable_col}' et/ou '{importance_col}' ne sont pas présentes dans le DataFrame.")

    # Trier par importance décroissante
    top_variables = importance_df.sort_values(by=importance_col, ascending=False).head(top_n)

    # Création du graphique
    plt.figure(figsize=(10, 8))
    plt.barh(top_variables[variable_col], top_variables[importance_col], color='steelblue')
    plt.xlabel("Importance")
    plt.ylabel("Variable")
    plt.title(f"Top {top_n} variables les plus importantes")
    plt.gca().invert_yaxis()  # Afficher les plus importantes en haut
    plt.show()

In [ ]:
# Visualisation des variables les plus importantes
plot_top_n_variables(
    importance_df = importance_df_with_labels,
    variable_col='LIB_VAR',
    importance_col='Importance',
    top_n=10
)